In [1]:
import numpy as np
# import pandas as pd

import torch 

import sys
sys.path.append('../')
sys.path.append('../../')

# 导入模型
from Networks.nn import NN_pricing_MGLU
from Pricing.rBergomi.rBergomi_utils import *
from calibrate_utils import *

# 评估使用 cpu
device = torch.device('cpu')

# 设置参数为 4 个
hyperparams = { 
    'input_dim': 4, 
    'hidden_dim': 64, 
    'hidden_nums': 4,
    'output_dim': 88
}

model = NN_pricing_MGLU(hyperparams=hyperparams).to(device=device, dtype=torch.float64)


model_state = torch.load( 
    r'../../Results/models/MGLU_rBergomi_flat_forward_variance.pth'
)
model.load_state_dict(model_state)

# 设置为 eval mode
model.eval()
model.to(device=device, dtype=torch.float64)

NN_pricing_MGLU(
  (layer_lst): ModuleList(
    (0): MGLUBlock(
      (mglu_projection): MGLU(in_features=4, out_features=64, bias=False)
    )
    (1-3): 3 x MGLUBlock(
      (mglu_projection): MGLU(in_features=64, out_features=64, bias=False)
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
  )
  (output_layer): Linear(in_features=64, out_features=88, bias=True)
)

In [2]:
# 数据集
import gzip
f = gzip.GzipFile(
    filename = r"../../Data/rBergomiTrainSet.txt.gz", 
    mode = "r"
)

data = np.load(f)
xx, yy = data[:, :4], data[:, 4:]

strikes=np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5 ])
maturities=np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0 ])

# xx: 参数
## 前 4 列代表网格所对应的参数
xx = data[:, :4]
print(f"参数形状：{xx.shape}")

# yy: 隐含波动率曲面 
# 后 88 列表示隐含波动率曲面 8 * 11 = 88
yy = data[:, 4:]
print(f"隐含波动率曲面形状：{yy.shape}")

# 参数
print(f"参数上界: {np.max(xx, axis=0)}")
print(f"参数下界: {np.min(xx, axis=0)}")


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append('../')

# from NN_Training.rBergomi_nn_pricer import x_transform, x_inv_transform, y_transform, y_inv_transform, params_scaler, params_inv_scaler


x_train, x_test, y_train, y_test = train_test_split(xx, yy, test_size=0.15, random_state=42)

scale_x, scale_y = StandardScaler(), StandardScaler()


# 工具函数——数据标准化
def x_transform(train_data, test_data): 
    return scale_x.fit_transform(train_data), scale_x.transform(test_data)

def x_inv_transform(x):
    return scale_x.inverse_transform(x)

def y_transform(train_data, test_data): 
    return scale_y.fit_transform(train_data), scale_y.transform(test_data)

def y_inv_transform(y):
    return scale_y.inverse_transform(y)


# 训练集的 Upper and Lower Bounds
upper_bound = np.array([0.16,4,-0.1,0.5])
lower_bound = np.array([0.01,0.3,-0.95,0.025])

def params_scaler(x): 
    return (x - (upper_bound+lower_bound) / 2 ) * 2 / (upper_bound-lower_bound)

def params_inv_scaler(x):
    return x * (upper_bound-lower_bound) / 2 + (upper_bound+lower_bound) / 2


x_train_transform = params_scaler(x_train)
x_test_transform = params_scaler(x_test)

y_train_transform, y_test_transform = y_transform(y_train, y_test)


train_data = (torch.from_numpy(x_train_transform).to(device=device),torch.from_numpy(y_train_transform).to(device=device))

test_data = (torch.from_numpy(x_test_transform).to(device=device),torch.from_numpy(y_test_transform).to(device=device))


print(f"训练集形状：{train_data[0].shape}")
print(f"测试集形状：{test_data[0].shape}")

参数形状：(40000, 4)
隐含波动率曲面形状：(40000, 88)
参数上界: [ 0.16      4.       -0.100011  0.499998]
参数下界: [ 0.0100133  0.300028  -0.949934   0.0250066]
训练集形状：torch.Size([34000, 4])
测试集形状：torch.Size([6000, 4])


In [7]:
from scipy.optimize import differential_evolution

def calibrate_with_de(model, market_iv, device, bounds):
    """
    使用差分进化算法校准模型参数。
    bounds: 参数上下界，例如 [(0.05,0.15), (1.5,3.0), (-0.9,-0.3), (0.02,0.2)]
    """
    model.eval()
    market_iv_tensor = torch.tensor(market_iv, device=device, dtype=torch.float64).flatten()
    
    # 目标函数
    def objective(params_np):
        params_torch = torch.tensor(params_np, dtype=torch.float64, device=device)
        with torch.no_grad():
            pred = model(params_torch.unsqueeze(0)).squeeze(0).flatten()
        loss = torch.nn.functional.mse_loss(pred, market_iv_tensor).item()
        return loss
    
    # 运行差分进化
    result = differential_evolution(
        func=objective,
        bounds=bounds,
        strategy='best1bin',      # 最常用的有效策略
        maxiter=200,              # 最大迭代数
        popsize=15,               # 种群大小，可调
        tol=1e-8,                 # 容差
        mutation=(0.5, 1.5),      # 差分缩放因子范围，动态更好
        recombination=0.7,        # 交叉概率
        seed=42,                  # 确保可复现
        disp=True,                # 显示进度
        workers=1                 # 可设为-1并行化（如果你的目标函数支持）
    )
    
    # print(f"DE优化成功: {result.success}")
    # print(f"最佳参数: {result.x}")
    # print(f"最佳损失: {result.fun:.6e}")
    # print(f"评估次数: {result.nfev}")
    
    return result.x, result.fun

# 使用示例
param_bounds = [(lower_bound[i], upper_bound[i]) for i in range(4)]
best_params, best_loss = calibrate_with_de(model, y_test_transform[0], device, param_bounds)

differential_evolution step 1: f(x)= 1.7733625013113865
differential_evolution step 2: f(x)= 1.693329458193096
differential_evolution step 3: f(x)= 1.693329458193096
differential_evolution step 4: f(x)= 1.6206954206437862
differential_evolution step 5: f(x)= 1.6206954206437862
differential_evolution step 6: f(x)= 1.5068284907098457
differential_evolution step 7: f(x)= 1.5059228031435126
differential_evolution step 8: f(x)= 1.4857556075371263
differential_evolution step 9: f(x)= 1.4857556075371263
differential_evolution step 10: f(x)= 1.3669561492303517
differential_evolution step 11: f(x)= 1.3669561492303517
differential_evolution step 12: f(x)= 1.3669561492303517
differential_evolution step 13: f(x)= 1.3669561492303517
differential_evolution step 14: f(x)= 1.3669561492303517
differential_evolution step 15: f(x)= 1.3669561492303517
differential_evolution step 16: f(x)= 1.3669561492303517
differential_evolution step 17: f(x)= 1.360042678587766
differential_evolution step 18: f(x)= 1.345

In [6]:
x_test_transform[0]

array([ 0.26272   , -0.85721946, -0.51287294, -0.07716211])